In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from src.PreProc_Data.DataProc import StackedSequenceDataset
import numpy as np
from src.MZA_Experiment import MZA_Experiment
# from MZA_Experiment import MZA_Experiment

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps") 
else:
    device = torch.device("cpu")

In [3]:
data = np.load("Data/2DCylinder/cyl_data_5_2.npy")
# data = data[:,:,:2]
# data = np.einsum('ijk->jki',data)
data.shape


(1, 201, 20354)

In [4]:
args_dict = {"device": device, "seq_len": 5}
dataset   = StackedSequenceDataset(data, args_dict)
it = iter(dataset)
Phi_seq, Phi_nn = next(it)
print(Phi_seq.shape)
print(Phi_nn.shape)
print(len(dataset))

torch.Size([5, 20354])
torch.Size([20354])
201


: 

In [4]:
seqdataset = SequenceDataset(data, device)
# dataloader = DataLoader(dataset, batch_size = 16, shuffle =True)

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



In [6]:
it = iter(seqdataset)
Phi_seq, Phi_nn = next(it)
for i, data in enumerate(seqdataset):
    if (i!=0):
        Phi_seq = torch.cat((Phi_seq, data[0]), dim = 0)
        Phi_nn  = torch.cat((Phi_nn, data[1]), dim = 0)
print(Phi_seq.shape)
print(Phi_nn.shape)
   


torch.Size([52920, 5, 2])
torch.Size([52920, 2])


In [7]:
dataloader = DataLoader(dataset, batch_size = 16, shuffle =True)

it = iter(dataloader)
Phi_seq, Phi_nn = next(it)
print(Phi_seq.shape)
print(Phi_nn.shape)

torch.Size([16, 5, 2])
torch.Size([16, 2])


In [1]:
k = {"a":1,"b":2}
print(type(k))

<class 'dict'>


In [3]:
from Layers.Koopman import Koopman


In [4]:
koopman = Koopman(5,device)
koopman.kMatrixUT.requires_grad 

True

In [5]:
sum(p.numel() for p in koopman.parameters())

15

In [6]:
for parameter in koopman.parameters():
    print(parameter)

Parameter containing:
tensor([0.5097, 0.2513, 0.4220, 0.8106, 0.9658], requires_grad=True)
Parameter containing:
tensor([-0.0132,  0.0109, -0.0005, -0.0084, -0.0013, -0.0097,  0.0081,  0.0172,
         0.0102, -0.0119], requires_grad=True)


In [10]:
import random
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

seq_len = 10
n_class = 5
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.W = nn.Parameter(torch.randn(seq_len, n_class), requires_grad=True)
        
    def forward(self, x):
        x = torch.matmul(x, self.W)
        x = F.relu(x)
        x = F.log_softmax(x)
        return x

model = SimpleNet()
print('parameters----------')
for parameter in model.parameters():
    print("hello")
    print(parameter)
    print("hello")

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.0001)
loss_fn = nn.NLLLoss()
for i in range(10000):
    in_data = Variable(torch.FloatTensor([[random.randint(0, n_class-1) for _ in range(seq_len)]]))
    target_data = Variable(torch.LongTensor([ int(in_data.data[0][0]) ]))
    
    optimizer.zero_grad()
    output = model(in_data)
    loss = loss_fn(output, target_data)
    loss.backward()
    optimizer.step()
    if i % 1000 == 0:
        print('loss:', loss.data[0])
    
# test
print('======Test======')
for i in range(10):
    in_data = Variable(torch.FloatTensor([[random.randint(0, n_class-1) for _ in range(seq_len)]]))
    target_data = int(in_data.data[0][0])
    out = model(in_data)
    _, ind = torch.max(out.data[0], 0)
    print(ind[0] == target_data, 'out', ind[0], 'label', target_data)

print('after parameters')
print(next(model.parameters()))

parameters----------
hello
Parameter containing:
tensor([[ 1.8619,  1.3373, -0.2634,  0.7723,  1.2466],
        [ 0.2350,  0.6656, -1.4673,  1.7389,  0.4045],
        [-1.2977,  1.6807,  0.4124,  1.7424, -1.0251],
        [ 0.3654, -0.5850,  0.5582,  1.3203,  0.2428],
        [-0.8385,  0.9671,  0.4986,  2.4612,  0.0583],
        [-1.0173, -1.2649,  0.8229,  1.5207, -0.7703],
        [-2.0080,  0.1509, -0.3980, -0.3626, -0.2976],
        [ 0.4990, -1.4490,  0.4683,  0.4888, -0.3042],
        [ 1.6785,  0.3926,  0.0045, -1.0586,  0.1292],
        [-0.5006,  0.5322, -0.7850,  0.3962,  0.2309]], requires_grad=True)
hello


/tmp/ipykernel_324/1620742790.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [3]:
import numpy as np
data = np.load("Data/KS/ks_N256_dt0.001_L6.npy")
data = data[:15,::100,:]
data.shape

(15, 10000, 256)

In [4]:
np.save("Data/KS/ks_N256_dt0.001_L6_short.npy", data)